In [9]:
import os, re
import pandas as pd
import numpy as np

# #显示所有列
# pd.set_option('display.max_columns', None)

# #显示所有行
# pd.set_option('display.max_rows', None)

pd.set_option('max_colwidth',100) 

In [10]:
folders = [
    # r'D:\code\forecast_model\notebook\20250811_base',
    # r'D:\code\forecast_model\notebook\20250811_small_large',
    # r'D:\code\forecast_model\notebook\20250811_extraLarge',
    # r"D:\code\forecast_model\notebook\20250815_base_large",
    # r'D:\code\forecast_model\notebook\20250817_base',
    # r'D:\code\forecast_model\notebook\20250820_base',
    # r'D:\code\forecast_model\notebook\20250820_bigger',
    # r'D:\code\forecast_model\notebook\20250821_mlp_base2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline3',
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline2",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline3",
    # r'D:\code\forecast_model\notebook\20250825_tcn_baseline',
    # r"D:\code\forecast_model\notebook\20250826_tcn_filt_days",
    # r'D:\code\forecast_model\notebook\20250828_tcn_data',
    # r'D:\code\forecast_model\notebook\20250829_tcn_data',
    # r"D:\code\forecast_model\notebook\20250829_tcn_data2",
    # r'D:\code\forecast_model\notebook\20250830_tcn_data',
    # r'D:\code\forecast_model\notebook\20250831_tcn_only30',
    # r'D:\code\forecast_model\notebook\20250831_nofilter_top5',
    # r'D:\code\forecast_model\notebook\20250831_tcn_we',
    # r'D:\code\forecast_model\notebook\20250831_tcn_we2',
    # r'D:\code\forecast_model\notebook\20250901_warinup',
    # r'D:\code\forecast_model\notebook\20250901_1_lr',
    # r'D:\code\forecast_model\notebook\20250901_2_smaller',
    # r'D:\code\forecast_model\notebook\20250901_3_no_nomovefilter',
    # r'D:\code\forecast_model\notebook\20250902_0_nomovefilter',
    # r'D:\code\forecast_model\notebook\20250902_1_dropout',
    # r'D:\code\forecast_model\notebook\20250902_3_smaller',
    r'D:\code\forecast_model\notebook\20250902_2_dropout2',
    ]

In [11]:
data = pd.DataFrame()
for folder in folders:
    for i in os.listdir(folder):
        if 'muilt_model' == i:continue
        # 20250811_base_P100_DeepLOB_v2_input_indepent	0.558035	2.77h
        # 20250811_base_P100_DeepLOB_v2_overfit
        if 'input_indepent' in i or 'overfit' in i: continue

        if i.endswith('.csv'): continue
        if i.endswith('.xlsx'): continue

        code = i.split('_')[3]
        fold = i.split('_')[4]

        train_folder = os.path.join(folder, i)
        if not os.path.isdir(train_folder): continue

        result_file = os.path.join(train_folder, 'result.csv')
        if not os.path.exists(result_file): continue

        # 修复替换
        _data = open(result_file, 'r').readlines()
        _data[1] = _data[1].replace('(30, 2, 1)', '30@2@1')
        with open(result_file, 'w') as f:
            f.writelines(_data)

        _data = pd.read_csv(result_file)
        # _data['describe.1'] = _data['describe.1'] + "_" + code + f"_fold{fold}"

        # 判断是否有nan
        if _data.isnull().values.any(): 
            print(i)
            continue

        if 'test_class_f1_0' in list(_data):
            _data[f'test_mean_class_f1'] = (_data[f'test_class_f1_0'] + _data[f'test_class_f1_1']) / 2
        elif 'test_final_class_f1_0' in list(_data):
            # 同时采用 best/final
            d2 = _data.copy()
            d2[f'test_mean_class_f1'] = _data[f'test_best_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                d2[f'test_mean_class_f1'] += _data[f'test_best_class_f1_{i}']
            d2[f'test_mean_class_f1'] = d2[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            d2['train_title'] += '_best'

            _data[f'test_mean_class_f1'] = _data[f'test_final_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                _data[f'test_mean_class_f1'] += _data[f'test_final_class_f1_{i}']
            _data[f'test_mean_class_f1'] = _data[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            _data['train_title'] += '_final'

            _data = pd.concat([_data, d2], ignore_index=True)

        # for _type in ['train', 'val', 'test_best', 'test_final', 'test_dummy']:
        #     _data[f'{_type}_mean_class_f1'] = (_data[f'{_type}_class_f1_0'] + _data[f'{_type}_class_f1_1']) / 2

        # for _type in ['train', 'val', 'test_best', 'test_final']:
            # _data[f'{_type}_mean_class_f1_enhanced_pct'] = 100 * (_data[f'{_type}_mean_class_f1'] - _data['test_dummy_mean_class_f1']) / _data['test_dummy_mean_class_f1']

        data = pd.concat([data, _data], ignore_index=True)

try:
    data['mean_test_01_f1'] = (data['test_best_class_f1_0'] + data['test_best_class_f1_1']) / 2
    data['mean_val_01_f1'] = (data['val_class_f1_0'] + data['val_class_f1_1']) / 2
except:
    pass

# 只使用 top 数据集 predict_n100
# data = data.loc[data['train_title'].str.contains('top5')].reset_index(drop=True)
# data = data.loc[data['train_title'].str.contains('predict_n100')].reset_index(drop=True)

data

,train_title,root,alist_upload_folder,amp,epochs,batch_n,batch_size,learning_rate,abs_learning_rate,no_better_stop,...,test_dummy_mean_class_f1,train_mean_class_f1_enhanced_pct,val_mean_class_f1_enhanced_pct,test_best_mean_class_f1_enhanced_pct,test_final_mean_class_f1_enhanced_pct,each_epoch_cost,cost,test_mean_class_f1,mean_test_01_f1,mean_val_01_f1
0,20250902_2_P100_x4_0.1_0.2_0.1_420_seed0_IDX0_final,20250902_2_P100_x4_0.1_0.2_0.1_420_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.495760,54.444685,47.393977,60.958999,54.175487,0.03h,5.89h,0.759800,0.797971,0.730721
1,20250902_2_P100_x4_0.1_0.2_0.1_420_seed0_IDX0_best,20250902_2_P100_x4_0.1_0.2_0.1_420_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.495760,54.444685,47.393977,60.958999,54.175487,0.03h,5.89h,0.787541,0.797971,0.730721
2,20250902_2_P100_x4_0.1_0.2_0.1_420_seed1_IDX3_final,20250902_2_P100_x4_0.1_0.2_0.1_420_seed1_IDX3,train_data,no,200,128,8192,0.0384,0,0,...,0.498073,53.340097,40.699110,53.907580,48.021231,0.03h,5.57h,0.733871,0.766572,0.700784
3,20250902_2_P100_x4_0.1_0.2_0.1_420_seed1_IDX3_best,20250902_2_P100_x4_0.1_0.2_0.1_420_seed1_IDX3,train_data,no,200,128,8192,0.0384,0,0,...,0.498073,53.340097,40.699110,53.907580,48.021231,0.03h,5.57h,0.762085,0.766572,0.700784
4,20250902_2_P100_x4_0.1_0.2_0.1_420_seed2_IDX6_final,20250902_2_P100_x4_0.1_0.2_0.1_420_seed2_IDX6,train_data,no,200,128,8192,0.0384,0,0,...,0.500748,52.257237,45.750649,60.174885,53.489001,0.03h,5.48h,0.763810,0.802073,0.729844
5,20250902_2_P100_x4_0.1_0.2_0.1_420_seed2_IDX6_best,20250902_2_P100_x4_0.1_0.2_0.1_420_seed2_IDX6,train_data,no,200,128,8192,0.0384,0,0,...,0.500748,52.257237,45.750649,60.174885,53.489001,0.03h,5.48h,0.795350,0.802073,0.729844
6,20250902_2_P100_x4_0.1_0.2_0.1_420_seed3_IDX9_final,20250902_2_P100_x4_0.1_0.2_0.1_420_seed3_IDX9,train_data,no,200,128,8192,0.0384,0,0,...,0.494975,54.235769,47.781878,59.485231,55.146219,0.03h,5.33h,0.770356,0.789411,0.731483
7,20250902_2_P100_x4_0.1_0.2_0.1_420_seed3_IDX9_best,20250902_2_P100_x4_0.1_0.2_0.1_420_seed3_IDX9,train_data,no,200,128,8192,0.0384,0,0,...,0.494975,54.235769,47.781878,59.485231,55.146219,0.03h,5.33h,0.777419,0.789411,0.731483
8,20250902_2_P100_x4_0.1_0.2_0.1_420_seed4_IDX12_final,20250902_2_P100_x4_0.1_0.2_0.1_420_seed4_IDX12,train_data,no,200,128,8192,0.0384,0,0,...,0.498735,53.339295,37.905998,53.567844,46.186572,0.03h,5.57h,0.732518,0.765896,0.687785
9,20250902_2_P100_x4_0.1_0.2_0.1_420_seed4_IDX12_best,20250902_2_P100_x4_0.1_0.2_0.1_420_seed4_IDX12,train_data,no,200,128,8192,0.0384,0,0,...,0.498735,53.339295,37.905998,53.567844,46.186572,0.03h,5.57h,0.763267,0.765896,0.687785


In [12]:
if len(folders) == 1:
    data.to_csv(os.path.join(folders[0], 'result.csv'), index=False)


In [13]:
list(data)

['train_title',
 'root',
 'alist_upload_folder',
 'amp',
 'epochs',
 'batch_n',
 'batch_size',
 'learning_rate',
 'abs_learning_rate',
 'no_better_stop',
 'checkpointing_steps',
 'label_smoothing',
 'weight_decay',
 'classify',
 'y_n',
 'debug',
 'test',
 'seed',
 'label_train',
 'label_val',
 'label_test_final',
 'model',
 'grad_norm_best',
 'grad_norm',
 'total_grad_norm_best',
 'total_grad_norm',
 'train_loss_best',
 'train_loss',
 'train_acc_best',
 'train_acc',
 'train_f1_best',
 'train_f1',
 'train_recall_best',
 'train_recall',
 'train_class_acc_0_best',
 'train_class_acc_0',
 'train_class_mcc_0_best',
 'train_class_mcc_0',
 'train_class_f1_0_best',
 'train_class_f1_0',
 'train_class_recall_0_best',
 'train_class_recall_0',
 'train_class_acc_1_best',
 'train_class_acc_1',
 'train_class_mcc_1_best',
 'train_class_mcc_1',
 'train_class_f1_1_best',
 'train_class_f1_1',
 'train_class_recall_1_best',
 'train_class_recall_1',
 'val_loss_best',
 'val_loss',
 'val_acc_best',
 'val_acc',

In [14]:
data[['train_title', 'label_train']]

,train_title,label_train
0,20250902_2_P100_x4_0.1_0.2_0.1_420_seed0_IDX0_final,1(364471)@1(364471)
1,20250902_2_P100_x4_0.1_0.2_0.1_420_seed0_IDX0_best,1(364471)@1(364471)
2,20250902_2_P100_x4_0.1_0.2_0.1_420_seed1_IDX3_final,1(364471)@1(364471)
3,20250902_2_P100_x4_0.1_0.2_0.1_420_seed1_IDX3_best,1(364471)@1(364471)
4,20250902_2_P100_x4_0.1_0.2_0.1_420_seed2_IDX6_final,1(364471)@1(364471)
5,20250902_2_P100_x4_0.1_0.2_0.1_420_seed2_IDX6_best,1(364471)@1(364471)
6,20250902_2_P100_x4_0.1_0.2_0.1_420_seed3_IDX9_final,1(364471)@1(364471)
7,20250902_2_P100_x4_0.1_0.2_0.1_420_seed3_IDX9_best,1(364471)@1(364471)
8,20250902_2_P100_x4_0.1_0.2_0.1_420_seed4_IDX12_final,1(364471)@1(364471)
9,20250902_2_P100_x4_0.1_0.2_0.1_420_seed4_IDX12_best,1(364471)@1(364471)


In [ ]:
# 多seed平均
key_col_name = ['train_f1']
key_col_name = ['train_loss', 'train_f1', 'val_f1', 'val_f1_best', 'val_loss', 'test_best_f1', 'label_train']
# key_col_name = ['train_loss', 'train_f1', 'val_f1', 'val_f1_best', 'val_loss', 'label_train', 'label_val']

d = data.loc[:, key_col_name + ['cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'seed' not in i]))
d['label_train'] = d['label_train'].apply(lambda x:int(x.split('@')[-1][2:-1])*2)
if 'label_val' in list(d):
    d['label_val'] = d['label_val'].apply(lambda x:int(x.split('@')[-1][2:-1])*2)

# 只考察 final
_match_bool = data['train_title'].apply(lambda x: 'final' in x or ('final' not in x and 'best' not in x))
d = d.loc[_match_bool]
# 去除 final/best 的标记
d['train_title'] = d['train_title'].apply(lambda x: x.replace('_final', '').replace('_best', ''))

# # 关键字过滤 白名单
# keys = ['bc']
# _match_key_bool = data['train_title'].apply(lambda x: any([k in x for k in keys]))
# d = d.loc[_match_key_bool]
# 关键字过滤 黑名单
keys = ['wd0.0001', 'wd0.001']
_match_key_bool = data['train_title'].apply(lambda x: all([k not in x for k in keys]))
d = d.loc[_match_key_bool]

# # 根据 idx 区分类别
# # 用于未区分title的情况
# _cls = d['train_title'].apply(lambda x: [(int(i.replace('IDX', '')) % 3) for i in x.split('_') if 'IDX' in i][0])
# _cls = _cls.apply(lambda x: ['nomove30', 'nomove50', 'nomove70'][x])
# d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i])) + '_' + _cls

# title 已经区分
d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(round(x,2)) + 'h')
d.sort_values('val_f1_best',ascending=False)


,train_loss,train_f1,val_f1,val_f1_best,val_loss,test_best_f1,label_train,cost
train_title,,,,,,,,
20250902_2_P100_x4_0.1_0.2_0.1_420,0.484696,0.764006,0.716561,0.747292,0.577086,0.784886,728942.0,5.57h
20250902_2_P100_x4_0.3_0.6_0.3_420,0.623330,0.654874,0.720268,0.745048,0.551011,0.785506,728942.0,5.78h
20250902_2_P100_x4_0.2_0.4_0.2_420,0.575144,0.698467,0.701546,0.729160,0.575325,0.765803,728942.0,5.61h


In [16]:
# 多IDX平均
key_col_name = 'val_f1'
key_col_name = 'label_train'

d = data.loc[:, [key_col_name, 'cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i and 'seed' not in i ]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values([key_col_name],ascending=False)

TypeError: agg function failed [how->mean,dtype->object]